## D5a example notebook
Example notebook of the D5a 16 channel, 18-bit DAC module. To use this, we need a D5a module and a controller module. This can be either the C1b/C2 combination or the C1.

To use the D5a module, we also need to import the SPI rack class. All communication with the SPI Rack will go through this object. Only one SPI_rack object can be active at a time on the PC. So before running another script, the connection in this one should be closed.

In [2]:
# Import SPI rack and D5a module
from spirack import SPI_rack, D5a_module

Open the SPI rack connection and unlock the controller. This is necessary after bootup of the controller module. If not unlocked, no communication with the modules can take place. The virtual COM port baud rate is irrelevant as it doesn't change the actual speed. Timeout can be changed, but 1 second is a good value.

In [4]:
COM_speed = 1e6 # Baud rate, doesn't matter much
timeout = 1 # In seconds

spi_rack = SPI_rack('COM4', COM_speed, timeout)
spi_rack.unlock() # Unlock the controller to be able to send data

Create a new D5a module object at the correct (set) module address using the SPI object. By default the module resets the output voltages to 0 Volt. Before it does this, it will read back the current value. If this value is non-zero it will slowly ramp it to zero. If `reset_voltages = False` then the output will not be changed. 

In [5]:
D5a = D5a_module(spi_rack, module=2, reset_voltages=True)

The output span of the DAC can be set in software to the following ranges:
* 0 to 4 Volt: range_4V_uni
* -4 to 4 Volt: range_4V_bi
* -2 to 2 Volt: range_2V_bi

By default it is set to 4V bipolar. Changing the span, determines the smallest step size possible. The DACs used are 18-bit, which gives a smallest step size of:

$$\text{LSB} = \frac{\text{Span}}{2^{18}}$$

In the case of a $\pm$2V span this gives a LSB of: ~15 $\mu$V. You can get the current stepsize of a certain DAC with the `get_stepsize` function.

In [14]:
D5a.change_span_update(1, D5a.range_2V_bi)

stepsize =  D5a.get_stepsize(2)
print("Stepsize: " + str(stepsize) + " V")

Stepsize: 1.52587890625e-05 V


The voltage can be set using the `set_voltage` function. If you want the output to be precisely equal to the set value, the voltage should be an integer multiple of the stepsize. Especially if sweeps are performed this is recommended. Otherwise the steps might nog be equidistant. Here we set the voltage of output 1 to 1.9 Volt. In the software the DAC count starts at 0, while on the front it starts at 1.

Optionally the output voltage might also be changed by setting a digital bit value ranging from 0 to $2^{18}-1$ using `change_value_update`.

In [10]:
# Changing the output by voltage
D5a.set_voltage(0, 1.9)

In [21]:
# Changing the output by ditigal values
D5a.change_value_update(1, 165535)

When done with the measurement, it is recommended to close the SPI Rack connection. This will allow other measurement scripts to acces the device.

In [22]:
spi_rack.close()